# **Lab: Model Optimization**



## Exercise 2: Xgboost with Hyperopt

We will train a Xgboost model on the same dataset as previously usiong Hyperopt.


**Pre-requisites:**
- Create a github account (https://github.com/join)
- Install git (https://git-scm.com/book/en/v2/Getting-Started-Installing-Git)
- Install Docker (https://docs.docker.com/get-docker/)

The steps are:
1.   Launch Docker image
2.   Load Data
3.   Train Xgboost model with defauly hyperparameter
4.   Hyperparameter tuning with Hyperopt
5.   Push changes


### 1. Launch Docker image

**[1.1]** Go to the folder you created previously `adv_dsi_lab_2`

In [ ]:
# Placeholder for student's code (command line)

In [3]:
# Solution:
cd ~/Users/anna/Desktop/ADV_DSI/adv_dsi_2023/adv_dsi_lab_2

NameError: name 'cd' is not defined

**[1.2]** Run the built Docker image

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution:
docker run  -dit --rm --name adv_dsi_lab_2 -p 8888:8888 -e JUPYTER_ENABLE_LAB=yes -v ~/Projects/adv_dsi_2023/adv_dsi_lab_2:/home/jovyan/work -v ~/Projects/adv_dsi_2023/.aws:/home/jovyan/.aws -v ~/Projects/adv_dsi_2023/src:/home/jovyan/work/src xgboost-notebook:latest 

Syntax: docker run [OPTIONS] IMAGE

Options:

`-dit: Run container in background and interactive`

`--rm: Automatically remove the container when it exits`

`--name: Assign a name to the container`

`-p: Publish a container's port(s) to the host`

`-e: Set environment variables`

`-v Bind mount a volume`

Documentation: https://docs.docker.com/engine/reference/commandline/run/

**[1.3]** Display last 50 lines of logs

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution
docker logs --tail 50 adv_dsi_lab_3

Syntax: docker logs [OPTIONS] CONTAINER

Options:

`--tail: Number of lines to show from the end of the logs`

Documentation: https://docs.docker.com/engine/reference/commandline/logs/

**[1.4]** Copy the url displayed and paste it to a browser in order to launch Jupyter Lab

**[1.5]** Create a new git branch called `xgboost_hyperopt`

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution
git checkout -b xgboost_hyperopt

Documentation: https://www.atlassian.com/git/tutorials/using-branches/git-checkout

**[1.6]** Navigate the folder `notebooks` and create a new jupyter notebook called `2_xgboost_hyperopt.ipynb`

### 2. Load Data

**[2.1]** Import the function you created `load_sets` from `src/data/sets`

In [ ]:
# Placeholder for student's code (Python code)

In [4]:
# Solution
from src.data.sets import load_sets

**[2.2]** Load the saved sets from `data/processed`

In [5]:
# Placeholder for student's code (Python code)

In [6]:
#Solution:
X_train, y_train, X_val, y_val, X_test, y_test = load_sets(path='../data/processed/')

# 3. Train Xgboost model

**[3.1]** Import the xgboost package as xgb


In [7]:
# Placeholder for student's code (Python code)

In [8]:
# Solution:
import xgboost as xgb

**[3.2]** Instantiate the RandomForest class into a variable called rf with random_state=8

In [ ]:
# Placeholder for student's code (Python code)

In [12]:
# Solution
xgboost1 = xgb.XGBClassifier(seed=42)

**[3.3]** Task: Fit the model with the prepared data

In [13]:
# Placeholder for student's code (Python code)

In [14]:
# Solution
xgboost1.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

**[3.4]** Import `dump` from `joblib` and save the fitted model into the folder `models` as a file called `xgboost_default`

In [15]:
# Placeholder for student's code (Python code)

In [16]:
# Solution:
from joblib import dump 

dump(xgboost1,  '../models/xgboost_default.joblib')

['../models/xgboost_default.joblib']

**[3.5]** Save the predictions from this model for the training and validation sets into 2 variables called `y_train_preds` and `y_val_preds`


In [17]:
# Placeholder for student's code (Python code)

In [18]:
# Solution:
y_train_preds = xgboost1.predict(X_train)
y_val_preds = xgboost1.predict(X_val)

**[3.6]** Import `print_reg_perf` from `src/models/performance` and display the accuracy and f1 scores of this baseline model on the training and validation sets

In [19]:
# Placeholder for student's code (Python code)

In [20]:
# Solution
from src.models.performance import print_class_perf

print_class_perf(y_preds=y_train_preds, y_actuals=y_train, set_name='Training', average='weighted')
print_class_perf(y_preds=y_val_preds, y_actuals=y_val, set_name='Validation', average='weighted')

Accuracy Training: 0.9241359991596344
F1 Training: 0.9238615052628725
Accuracy Validation: 0.9064686232904052
F1 Validation: 0.9059566769889884


#4. Hyperparameter tuning with Hyperopt

**[4.1]** Import Trials, STATUS_OK, tpe, hp, fmin from hyperopt package

In [21]:
# Placeholder for student's code (Python code)

In [22]:
# Solution:
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

**[4.2]** Define the search space for xgboost hyperparameters

In [23]:
space = {
    'max_depth' : hp.choice('max_depth', range(5, 8, 1)),
    'min_child_weight' : hp.choice('min_child_weight', range(3, 6, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.4, 0.05),    
    'subsample' : hp.quniform('subsample', 0.7, 1, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.2, 0.5, 0.05)
}

**[4.3]** Define a function called `objective` with the following logics:
- input parameters: hyperparameter seacrh space (`space`)
- logics: train a xgboost model with the search space and calculate the average accuracy score for cross validation with 5 folds
- output parameters: dictionary with the loss score and STATUS_OK

In [24]:
# Placeholder for student's code (Python code)

In [25]:
# Solution:
def objective(space):
    from sklearn.model_selection import cross_val_score
    
    xgboost = xgb.XGBClassifier(
        max_depth = int(space['max_depth']),
        learning_rate = space['learning_rate'],
        min_child_weight = space['min_child_weight'],
        subsample = space['subsample'],
        colsample_bytree = space['colsample_bytree']
    )
    
    acc = cross_val_score(xgboost, X_train, y_train, cv=5, scoring="accuracy").mean()

    return{'loss': 1-acc, 'status': STATUS_OK }

**[4.4]** Launch Hyperopt search and save the result in a variable called `best`

In [26]:
best = fmin(
    fn=objective,   
    space=space,       
    algo=tpe.suggest,       
    max_evals=3
)

100%|██████████| 3/3 [02:34<00:00, 51.55s/trial, best loss: 0.09560558843096756]


**[4.5]** Print the best set of hyperparameters

In [27]:
# Placeholder for student's code (Python code)

In [28]:
# Solution:
print("Best: ", best)

Best:  {'colsample_bytree': 0.45, 'learning_rate': 0.35000000000000003, 'max_depth': 1, 'min_child_weight': 2, 'subsample': 0.8}


**[4.6]** Instantiate a XGBClassifier with best set of hyperparameters

In [29]:
# Placeholder for student's code (Python code)

In [30]:
# Solution:
xgboost2 = xgb.XGBClassifier(
    max_depth = best['max_depth'],
    learning_rate = best['learning_rate'],
    min_child_weight = best['min_child_weight'],
    subsample = best['subsample'],
    colsample_bytree = best['colsample_bytree']
)

**[4.7]** Fit the model with the prepared data

In [31]:
# Placeholder for student's code (Python code)

In [32]:
# Solution:
xgboost2.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.45, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.35000000000000003,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

**[4.8]** Display the accuracy and f1 scores of this baseline model on the training and validation sets

In [ ]:
# Placeholder for student's code (Python code)

In [ ]:
# Solution:
print_class_perf(y_preds=xgboost2.predict(X_train), y_actuals=y_train, set_name='Training', average='weighted')
print_class_perf(y_preds=xgboost2.predict(X_val), y_actuals=y_val, set_name='Validation', average='weighted')

Accuracy Training: 0.8582093210546587
F1 Training: 0.8561802532165359
Accuracy Validation: 0.859639908401437
F1 Validation: 0.8574571320912837


**[4.9]** Save the fitted model into the folder models as a file called `xgboost_best`

In [ ]:
# Placeholder for student's code (Python code)

In [ ]:
# Solution:
dump(xgboost2,  '../models/xgboost_best.joblib')

['../models/xgboost_best.joblib']

# 5.   Push changes

**[5.1]** Add you changes to git staging area

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution:
git add .

**[5.2]** Create the snapshot of your repository and add a description

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution:
git commit -m "xgboost hyperopt"

**[5.3]** Push your snapshot to Github


In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution:
git push

**[5.4]** Go to Github and merge the branch after reviewing the code and fixing any conflict




**[5.5]** Check out to the master branch

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution:
git checkout master

**[5.6]** Pull the latest updates

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution
git pull

**[5.7]** Stop the Docker container

In [ ]:
# Placeholder for student's code (command line)

In [ ]:
# Solution
docker stop adv_dsi_lab_2

Documentation: https://docs.docker.com/engine/reference/commandline/stop/